In [326]:
## Main SWR detection analysis! Note there are two ways to analyze the data: a cluster and local version. I keep them
## both updated with changes for now since I tend to check with the local version even if the cluster version is better
## for bulk analysis 2020-6-17 JS

In [1]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)
exp = 'FR1'
poss_subs = df[df.experiment == exp]

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [47]:
# sub lists I've been using for sample code

# subs = ['R1002P','R1003P','R1006P','R1010J','R1112M','R1163T'] # initial 6 for hippocampus
# subs = subs+['R1001P','R1092J','R1151E','R1320D'] # additional subs with issues that I resolved with various RR and corr thresholds
# subs = ['R1112M','R1018P','R1020J','R1022J','R1023J','R1026D','R1027J'] # 7 with entorhinal
# subs = ['R1010J','R1112M'] # original 2
# subs = ['R1112M','R1163T'] #'R1112M' # subjects with FR1 and catFR1. SRs of 1000 and 500 tho
# subs = ['R1151E'] # all channels X sessions get removed but 3 after 60/180 hz line removal
# these should all be worked out now. See SWR FR1 problem sessions PPT on Box for details of each
# subs = ['R1299T','R1332M','R1397D','R1349T','R1339D','R1337E','R1275D','R1151E','R1123C','R1120E','R1045E'] # final surrounding_recall problem subs after looking at huge raster!
# subs = ['R1308T','R1338T','R1358T'] # these guys had weird SRs and some loading problems when I went to whole_retrieval analysis...fixed with rounding
# subs = ['R1023J','R1101T','R1148P','R1368T','R1323T','R1334T'] # single session subs with memory allocation errors in ClusterRun
# subs = ['R1006P','R1010J','R1030J','R1032D','R1049J','R1051J','R1052E','R1054J','R1093J','R1098D','R1104D','R1108J','R1111M',
#         'R1115T','R1118N','R1124J','R1207J','R1230J','R1316T','R1329T','R1323T','R1337E','R1367D'] # subs that didn't load on 2020-07-04 encoding run
# subs = ['R1368T','R1461T','R1463E'] # subs with particularly low localization.pair matches in pairs...see Excel in loading info on Box for details
subs = ['R1433E','R1355T','R1367D','R1368T'] # subs with "ca3" or "dg"

bad_subs = ['R1433E','R1051J'] # no electrode cats

sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp)] # all sessions for subs
# sub_df = df[(df.subject.isin(subs))  & (df.experiment == exp) & (df.session==1)] # single session
# sub_df = df[(df.experiment == exp) & (df.session == 0)] # all FR subs 
# sub_df = sub_df[1:2]
sub_df

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
256,NaN,protocols/r1/subjects/R1051J/experiments/FR1/s...,protocols/r1/subjects/R1051J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1051J/experiments/FR1/s...,0,NaN,0,protocols/r1/subjects/R1051J/localizations/0/m...,NaN,0,R1051J,R1051J,NaN,protocols/r1/subjects/R1051J/experiments/FR1/s...
257,NaN,protocols/r1/subjects/R1051J/experiments/FR1/s...,protocols/r1/subjects/R1051J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1051J/experiments/FR1/s...,0,NaN,1,protocols/r1/subjects/R1051J/localizations/0/m...,NaN,1,R1051J,R1051J,NaN,protocols/r1/subjects/R1051J/experiments/FR1/s...
258,NaN,protocols/r1/subjects/R1051J/experiments/FR1/s...,protocols/r1/subjects/R1051J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1051J/experiments/FR1/s...,0,NaN,2,protocols/r1/subjects/R1051J/localizations/0/m...,NaN,2,R1051J,R1051J,NaN,protocols/r1/subjects/R1051J/experiments/FR1/s...
1802,NaN,protocols/r1/subjects/R1355T/experiments/FR1/s...,protocols/r1/subjects/R1355T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1355T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1355T/localizations/0/m...,NaN,0,R1355T,R1355T,3.1,protocols/r1/subjects/R1355T/experiments/FR1/s...
1803,NaN,protocols/r1/subjects/R1355T/experiments/FR1/s...,protocols/r1/subjects/R1355T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1355T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1355T/localizations/0/m...,NaN,1,R1355T,R1355T,3.1,protocols/r1/subjects/R1355T/experiments/FR1/s...
1804,NaN,protocols/r1/subjects/R1355T/experiments/FR1/s...,protocols/r1/subjects/R1355T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1355T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1355T/localizations/0/m...,NaN,2,R1355T,R1355T,3.1,protocols/r1/subjects/R1355T/experiments/FR1/s...
1824,NaN,protocols/r1/subjects/R1367D/experiments/FR1/s...,protocols/r1/subjects/R1367D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1367D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1367D/localizations/0/m...,NaN,0,R1367D,R1367D,3.1,protocols/r1/subjects/R1367D/experiments/FR1/s...
1825,NaN,protocols/r1/subjects/R1367D/experiments/FR1/s...,protocols/r1/subjects/R1367D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1367D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1367D/localizations/0/m...,NaN,1,R1367D,R1367D,3.1,protocols/r1/subjects/R1367D/experiments/FR1/s...
1827,NaN,protocols/r1/subjects/R1368T/experiments/FR1/s...,protocols/r1/subjects/R1368T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1368T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1368T/localizations/0/m...,NaN,0,R1368T,R1368T,3.3,protocols/r1/subjects/R1368T/experiments/FR1/s...
2064,NaN,protocols/r1/subjects/R1433E/experiments/FR1/s...,protocols/r1/subjects/R1433E/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1433E/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1433E/localizations/0/m...,NaN,0,R1433E,R1433E,3.4,protocols/r1/subjects/R1433E/experiments/FR1/s...


In [44]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
exp_df = exp_df[
                ((df.subject!='R1015J') | (df.session!=0)) & 
                ((df.subject!='R1063C') | (df.session!=1)) & 
                ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                ((df.subject!='R1120E') | (df.session!=0)) &
                ((df.subject!='R1122E') | (df.session!=2)) &
                ((df.subject!='R1154D') | (df.session!=0)) &
                ((df.subject!='R1186P') | (df.session!=0)) &
                ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                ((df.subject!='R1277J') | (df.session!=0)) &
                ((df.subject!='R1413D') | (df.session!=0)) & 
                ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                ((df.subject!='R1275D') | (df.session!=3)) # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
               ] 
exp_df.shape
exp_df = exp_df[419:] # if want to see sessions with localization.json only
exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(553, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
1761,NaN,protocols/r1/subjects/R1345D/experiments/FR1/s...,protocols/r1/subjects/R1345D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1345D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1345D/localizations/0/m...,NaN,0,R1345D,R1345D,3.1,protocols/r1/subjects/R1345D/experiments/FR1/s...
1762,NaN,protocols/r1/subjects/R1345D/experiments/FR1/s...,protocols/r1/subjects/R1345D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1345D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1345D/localizations/0/m...,NaN,1,R1345D,R1345D,3.1,protocols/r1/subjects/R1345D/experiments/FR1/s...
1763,NaN,protocols/r1/subjects/R1345D/experiments/FR1/s...,protocols/r1/subjects/R1345D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1345D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1345D/localizations/0/m...,NaN,2,R1345D,R1345D,3.1,protocols/r1/subjects/R1345D/experiments/FR1/s...
1764,NaN,protocols/r1/subjects/R1345D/experiments/FR1/s...,protocols/r1/subjects/R1345D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1345D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1345D/localizations/0/m...,NaN,3,R1345D,R1345D,3.1,protocols/r1/subjects/R1345D/experiments/FR1/s...
1765,NaN,protocols/r1/subjects/R1345D/experiments/FR1/s...,protocols/r1/subjects/R1345D/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1345D/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1345D/localizations/0/m...,NaN,4,R1345D,R1345D,3.1,protocols/r1/subjects/R1345D/experiments/FR1/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,NaN,protocols/r1/subjects/R1462M/experiments/FR1/s...,protocols/r1/subjects/R1462M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1462M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1462M/localizations/0/m...,NaN,0,R1462M,R1462M,3.4,protocols/r1/subjects/R1462M/experiments/FR1/s...
2149,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,0,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,0,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2150,NaN,protocols/r1/subjects/R1463E/experiments/FR1/s...,protocols/r1/subjects/R1463E/localizations/1/m...,FR1,build,1,protocols/r1/subjects/R1463E/experiments/FR1/s...,1,NaN,1,protocols/r1/subjects/R1463E/localizations/1/m...,NaN,1,R1463E,R1463E_1,3.4,protocols/r1/subjects/R1463E/experiments/FR1/s...
2164,NaN,protocols/r1/subjects/R1466J/experiments/FR1/s...,protocols/r1/subjects/R1466J/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1466J/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1466J/localizations/0/m...,NaN,0,R1466J,R1466J,3.4,protocols/r1/subjects/R1466J/experiments/FR1/s...


## LOCAL run (original program)

In [48]:
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload

### PARAMS ###

save_values = 1

selected_period = 'surrounding_recall' # surrounding_recall # whole_retrieval # encoding 
# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
recall_type_switch = 0 # how do we select recalls?? Numbers 0:3
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur
# 2: test condition where we look at second recalls within IRI ONLY
# 3: ISOLATED only!

selected_region = HPC_labels #HPC_labels # ENT_labels+HPC_labels 

remove_soz_ictal = False # False only removes bad electrodes and not soz/ictal
min_ripple_rate = 0.1 # Hz.
max_ripple_rate = 1.0 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2 #???

filter_type = 'hamming' # 'butter' # Vaz used Butterworth from 80-120 Hz; Norman used Hamming from 70-180 Hz
# •Butterworth from Vaz et al: 2nd order from 80-120 ms, hilbert, select events >2 STD above mean of filtered traces.
#      Events >=25 ms long w/ max amp >3 SD were retained as ripples. Join adjacent ripples separated by <15 ms 
# •Hilbert from Norman et al: "70-180 Hz zero-lag linear-phase Hamming FIR filter w/ transition bandwidth of 5 Hz"
#      Then Hilbert, clip extreme to 4 SD, square this clipped, smooth w/ Kaiser FIR low-pass filter with 40 Hz cutoff,
#      mean and SD computed across entire experimental duration to define the threshold for event detection
#      Events from original (squared but unclipped) signal >4 SD above baseline were selected as candidate SWR events. 
#      Duration expanded until ripple power <2 SD. Events <20 ms or >200 ms excluded. Adjacent events <30 ms separation (peak-to-peak) merged.
# •Using IED detection from Vaz for 250 Hz highpass events (really 250-499). Norman uses 20-60 Hz events which is...odd.
#       See detectRipples code in module for this algorithm


# recall params
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
retrieval_whole_time = 30000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 300 # buffer to add to either end of IRI when processing eeg #**

### END PARAMS ###

if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
if selected_period == 'surrounding_recall':
    psth_start = -IRI # only makes sense to look at period <= IRI
    psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time

ripple_array = []; fr_array = []; 
trial_nums = []; 
session_ct = 0; channel_ct = 0; total_channel_ct = 0
HPC_names = []; sub_names = []; sub_sess_names = []
electrodes_per_session = []
total_lists = 0; total_recalls = 0; kept_recalls = 0
ent_elec_ct = []; sd_regions = []; not_sd_regions = []
ripple_ied_accum_ct = []
time_add_save = [];             
encoded_word_key_array = []; serialpos_array = [] # encoding info
ordinal_position_array = []

trial_by_trial_correlation = []; elec_by_elec_correlation = []
elec_ripple_rate_array = []

all_regions = []
all_atlas = []
all_pair_numbers = []
all_stein_das = []
non_locs = 0

for row in sub_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')
        evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.recalled==True)] # recalled word AND correct (from this list...no instrusions). 
        
        # need to remove free recalls that happened more than once        
        word_evs = evs[evs['type']=='WORD'] # get words 
        evs_free_recall = removeRepeatedRecalls(evs_free_recall,word_evs)
        
        # select which recalls??         
        [recall_selection_name,selected_recalls_idxs] = selectRecallType(recall_type_switch,evs_free_recall,IRI)
        evs_free_recall = evs_free_recall[selected_recalls_idxs]
        
        # let's make sure remaining recalls have eeg 
        evs_free_recall = evs_free_recall[evs_free_recall.eegoffset > -1]
        if len(evs_free_recall)==0: #...and that any are left!
            break
            
        # get output position in each list for this session's recalls
        session_corrected_list_ops = getOutputPositions(evs,evs_free_recall)
            
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
        except:
            localizations = []
            non_locs+=1
            ## test
        if len(localizations) > 1:
            loc_translation = Loc2PairsTranslation(pairs,localizations)
            loc_pairs = localizations.type.pairs
            num_translations = sum([loc!=' ' for loc in loc_translation])
            if len(pairs) >= len(loc_pairs):
                if num_translations != len(loc_pairs):
                    print(sub+'-'+str(session)+'-more pairs; len(pairs): '+str(len(pairs))+', len(loc_pairs): '+str(len(loc_pairs))+', trans: '+str(num_translations))
            elif len(pairs) < len(loc_pairs):
                if num_translations != len(pairs):
                    print(sub+'-'+str(session)+'-more loc_pairs; len(loc_pairs): '+str(len(loc_pairs))+', len(pairs): '+str(len(pairs))+', trans: '+str(num_translations))
        
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(tal_struct,localizations,pairs) 
        elec_regions
        
#         import ipdb; ipdb.set_trace()
        # to view all loc_pairs at once: [loc_pairs[lp:lp+1] for lp in range(len(loc_pairs)-1)]
        # to view all pairs at once: [pairs[p:p+1].label for p in range(len(pairs)-1)]
        
        all_regions.extend(elec_regions)
        all_atlas.extend(atlas_type)
        all_pair_numbers.extend(pair_number)
        all_stein_das.append(has_stein_das)
        
        sub_sess_names.append(sub+'-'+str(session))
            
    except Exception as e:
        LogDFExceptionLine(row, e, 'SWR_get_eeg_log.txt') #np.save('SWRscript_FR1.npy', ['Exception', str(e.__class__), str(e)])
        continue



/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1051J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1051J/tal/R1051J_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could not load MNI coordinates
  "Could not load MNI coordinates"))
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)


array(['superiorparietal', 'superiorparietal', 'postcentral',
       'precentral', 'precentral', 'superiorfrontal', 'superiorfrontal',
       'inferiorparietal', 'superiorparietal', 'postcentral',
       'postcentral', 'precentral', 'precentral', 'caudalmiddlefrontal',
       'rostralmiddlefrontal', 'rostralmiddlefrontal',
       'caudalmiddlefrontal', 'rostralmiddlefrontal',
       'rostralmiddlefrontal', 'caudalmiddlefrontal', 'paracentral',
       'paracentral', 'paracentral', 'precentral', 'precentral',
       'precentral', 'superiorfrontal', 'left precentral gyrus',
       'isthmuscingulate', 'posteriorcingulate', 'posteriorcingulate',
       'posteriorcingulate', 'posteriorcingulate',
       'left precentral gyrus', 'posteriorcingulate', 'superiorfrontal',
       'superiorfrontal', 'superiorfrontal', 'posteriorcingulate',
       'posteriorcingulate', 'superiorfrontal', 'superiorfrontal',
       'superiorfrontal', 'posteriorcingulate', 'isthmuscingulate',
       'posteriorcingulat

/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1051J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1051J/tal/R1051J_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could not load MNI coordinates
  "Could not load MNI coordinates"))
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)


array(['superiorparietal', 'superiorparietal', 'postcentral',
       'precentral', 'precentral', 'superiorfrontal', 'superiorfrontal',
       'inferiorparietal', 'superiorparietal', 'postcentral',
       'postcentral', 'precentral', 'precentral', 'caudalmiddlefrontal',
       'rostralmiddlefrontal', 'rostralmiddlefrontal',
       'caudalmiddlefrontal', 'rostralmiddlefrontal',
       'rostralmiddlefrontal', 'caudalmiddlefrontal', 'paracentral',
       'paracentral', 'paracentral', 'precentral', 'precentral',
       'precentral', 'superiorfrontal', 'left precentral gyrus',
       'isthmuscingulate', 'posteriorcingulate', 'posteriorcingulate',
       'posteriorcingulate', 'posteriorcingulate',
       'left precentral gyrus', 'posteriorcingulate', 'superiorfrontal',
       'superiorfrontal', 'superiorfrontal', 'posteriorcingulate',
       'posteriorcingulate', 'superiorfrontal', 'superiorfrontal',
       'superiorfrontal', 'posteriorcingulate', 'isthmuscingulate',
       'posteriorcingulat

/home1/john/Downloads/cmlreaders-master/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1051J/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1051J/tal/R1051J_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/john/Downloads/cmlreaders-master/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could not load MNI coordinates
  "Could not load MNI coordinates"))
/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/readers/tal.py:43: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)


...

array([' left gre gyrus rectus', ' left gre gyrus rectus',
       ' left cerebral white matter', ' left cerebral white matter',
       ' left cerebral white matter', ' left cerebral white matter',
       ' left cerebral white matter', ' left cerebral white matter',
       ' left cerebral white matter', ' left cerebral white matter',
       ' left mfg middle frontal gyrus', ' left mfg middle frontal gyrus',
       ' left cerebral white matter', ' left cerebral white matter',
       ' left mfg middle frontal gyrus',
       ' right morg medial orbital gyrus',
       ' right morg medial orbital gyrus',
       ' right morg medial orbital gyrus',
       ' right morg medial orbital gyrus',
       ' right morg medial orbital gyrus',
       ' right aorg anterior orbital gyrus',
       ' right morg medial orbital gyrus',
       ' right aorg anterior orbital gyrus',
       ' right cerebral white matter', ' right cerebral white matter',
       ' right cerebral white matter', ' right mfg middle fro

R1368T-0-more loc_pairs; len(loc_pairs): 194, len(pairs): 132, trans: 81


array(['"ca1"', ' right cerebral white matter', 'No atlas',
       ' right stg superior temporal gyrus',
       ' right stg superior temporal gyrus',
       ' right stg superior temporal gyrus',
       ' right stg superior temporal gyrus', 'No atlas',
       ' right cerebral white matter', ' right putamen',
       ' right cerebral white matter', 'No atlas',
       ' right cerebral white matter', ' right cerebral white matter',
       ' right stg superior temporal gyrus', 'No atlas', ' right putamen',
       ' right cerebral white matter', ' right cerebral white matter',
       'No atlas', 'No atlas', ' right tmp temporal pole', 'No atlas',
       ' right pins posterior insula', 'superiortemporal',
       ' right co central operculum', 'No atlas',
       ' right prg precentral gyrus', 'No atlas',
       ' right phg parahippocampal gyrus', ' right cerebral white matter',
       'No atlas', ' right cerebral white matter', 'No atlas',
       ' right stg superior temporal gyrus',
       ' r

R1433E-0-more pairs; len(pairs): 148, len(loc_pairs): 135, trans: 131


array([' left hippocampus', ' left hippocampus', ' left hippocampus',
       ' left inf lat vent', ' left cerebral white matter',
       ' left cerebral white matter', ' left mtg middle temporal gyrus',
       ' left mtg middle temporal gyrus',
       ' left mtg middle temporal gyrus', 'No atlas',
       ' left cerebral white matter', ' left hippocampus',
       ' left hippocampus', ' left cerebral white matter',
       ' left fug fusiform gyrus', ' left cerebral white matter',
       ' left cerebral white matter', ' left mtg middle temporal gyrus',
       ' left mtg middle temporal gyrus', 'No atlas',
       ' left thalamus proper', ' left hippocampus', ' left hippocampus',
       ' left hippocampus', ' left cerebral white matter',
       ' left cerebral white matter', ' left cerebral white matter',
       ' left cerebral white matter', ' left cerebral white matter',
       'No atlas', ' left cerebral white matter',
       ' left cerebral white matter', ' left cerebral white matter',


In [34]:
len(all_atlas)
len(all_regions)
# np.unique(all_atlas)
# np.unique(all_regions)
len(all_stein_das)

# what sessions have stein and das annotated?
np.array(all_stein_das)

# how often is "MTL" atlas used?
np.array(all_regions)[np.array(all_atlas)=='MTL_localization']

# what % of sessions have localization.json?
(len(exp_df)-non_locs)/len(exp_df)*100

65472

65472

548

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

array(['"sub"', '"ba36"', '"ba36"', '"ca1"', '"ba36"', '"sub"', '"ba36"',
       '"ba36"', '"ca1"', '"ba36"', '"ca1"', '"ca1"', '"ca1"', '"ca1"',
       '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"',
       '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"',
       '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"',
       '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"ca1"', '"misc"', '"dg"',
       '"dg"', '"misc"', '"dg"', '"dg"', '"misc"', '"dg"', '"dg"',
       '"ca1"', '"dg"', '"dg"', '"dg"', '"ca1"', '"dg"', '"dg"', '"dg"',
       '"dg"', '"dg"', '"ca1"', '"dg"', '"sub"', '"ca1"', '"dg"', '"dg"',
       '"ca1"', '"dg"', '"sub"', '"ca1"', '"dg"', '"dg"', '"ca1"', '"dg"',
       '"sub"', '"ca1"', '"ca1"', '"dg"', '"ca1"', '"ba36"', '"ba36"',
       '"ca1"', '"ba36"', '"ca1"', '"ca1"', '"ba36"', '"ba36"', '"ba36"',
       '"ca1"', '"ba36"', '"ca1"', '"ca1"', '"ba36"', '"ba36"', '"ba36"',
       '"ba36"', '"ca1"', '"dg"', '"dg"', '"ca1"', '"ba36"', '"ba3

24.95479204339964

In [35]:
sum(np.array(all_atlas)=='MTL_localization')

335

In [36]:
np.unique(all_atlas)

array(['MTL_localization', 'No atlas', 'das', 'dk', 'dk_localization',
       'ind', 'stein', 'wb', 'wb_localization'], dtype='<U16')

In [37]:
np.unique(all_regions)

array([' left acgg anterior cingulate gyrus',
       ' left ains anterior insula', ' left amygdala',
       ' left ang angular gyrus', ' left aorg anterior orbital gyrus',
       ' left calc calcarine cortex', ' left caudate',
       ' left cerebellum exterior', ' left cerebral white matter',
       ' left co central operculum', ' left cun cuneus',
       ' left ent entorhinal area', ' left fo frontal operculum',
       ' left frp frontal pole', ' left fug fusiform gyrus',
       ' left gre gyrus rectus', ' left hippocampus',
       ' left inf lat vent', ' left iog inferior occipital gyrus',
       ' left itg inferior temporal gyrus', ' left lateral ventricle',
       ' left lig lingual gyrus', ' left lorg lateral orbital gyrus',
       ' left mcgg middle cingulate gyrus',
       ' left mfc medial frontal cortex',
       ' left mfg middle frontal gyrus',
       ' left mog middle occipital gyrus',
       ' left morg medial orbital gyrus',
       ' left mpog postcentral gyrus medial segm